In [1]:
'''FINAL-SIS420'''
'''Taboada Rodas Oscar'''
'''One vs all'''

'One vs all'

In [2]:
import os
import numpy as np
from matplotlib import pyplot
from scipy import optimize

import pandas as pd

%matplotlib inline


In [3]:
# data = np.loadtxt(os.path.join('7. Internet_ads', 'Anuncios_internet.csv'), delimiter = ',')
data = np.loadtxt(os.path.join('DV.csv'), delimiter = ',')
# data = pd.read_csv('https://query.data.world/s/yzstkado34nyn736kvzkiyztebzyzt', low_memory=False)

data.shape

(935, 18)

In [4]:
X = data[:, :15]
y = data[:, 17]

print('X \n',X,'\n y\n',y)


X 
 [[1.200e+01 2.017e+03 5.000e+00 ... 1.000e+00 1.000e+00 2.000e+00]
 [1.200e+01 2.013e+03 4.000e+00 ... 1.000e+00 1.000e+00 2.000e+00]
 [1.900e+01 2.019e+03 5.000e+00 ... 1.000e+00 1.000e+00 2.000e+00]
 ...
 [1.700e+01 1.992e+03 5.000e+00 ... 2.000e+00 1.000e+00 2.000e+00]
 [1.300e+01 2.012e+03 5.000e+00 ... 2.000e+00 1.000e+00 2.000e+00]
 [4.000e+00 2.020e+03 4.000e+00 ... 1.000e+00 1.000e+00 2.000e+00]] 
 y
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1.
 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

In [5]:
X.shape[1]

15

In [6]:
input_layer_size  = 15
num_labels = 2
m = y.size

In [7]:
# def calcularSigmoide(z):
#     # Calcula la sigmoide de z.

#     return 1.0 / (1.0 + np.exp(-z))
from scipy.special import expit
def calcularSigmoide(z):
    return expit(z)


In [8]:
def calcularCosto(theta, X, y, lambda_):
    # Inicializa algunos valores utiles
    m = y.size
    
    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)
    
    J = 0
    grad = np.zeros(theta.shape)
    
    h = calcularSigmoide(X.dot(theta.T))
    
    temp = theta
    temp[0] = 0
    
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
    
    grad = (1 / m) * (h - y).dot(X)
    # Se aplica regularizacion en la siguiente linea 
    grad = grad + (lambda_ / m) * temp

    return J, grad

In [9]:
# valores de prueba para los parámetros theta
theta_t = np.array([-2, -1, 1, 2], dtype=float)

# valores de prueba para las entradas
X_t = np.concatenate((np.ones((5, 1)), np.arange(1, 16).reshape(5, 3, order='F')/10.0), axis=1)
print(X_t)
# valores de testeo para las etiquetas
y_t = np.array([1, 0, 1, 0, 1])

# valores de testeo para el parametro de regularizacion
lambda_t = 2

[[1.  0.1 0.6 1.1]
 [1.  0.2 0.7 1.2]
 [1.  0.3 0.8 1.3]
 [1.  0.4 0.9 1.4]
 [1.  0.5 1.  1.5]]


In [10]:
J, grad = calcularCosto(theta_t, X_t, y_t, lambda_t)

print('Costo         : {:.6f}'.format(J))
print('Costo esperadot: 2.534819')
print('-----------------------')
print('Gradientes:')
print(' [{:.6f}, {:.6f}, {:.6f}, {:.6f}]'.format(*grad))
print('Gradientes esperados:')
print(' [0.146561, -0.548558, 0.724722, 1.398003]');

Costo         : 1.934819
Costo esperadot: 2.534819
-----------------------
Gradientes:
 [0.146561, -0.348558, 0.524722, 0.998003]
Gradientes esperados:
 [0.146561, -0.548558, 0.724722, 1.398003]


In [11]:
def oneVsAll(X, y, num_labels, lambda_):
    
    m, n = X.shape
    
    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)


    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(calcularCosto, 
                                initial_theta, 
                                (X, (y == (c + 1)), lambda_), 
                                jac=True, 
                                method='BFGS',
                                options=options) 
        all_theta[c] = res.x

    return all_theta

In [12]:
lambda_ = 0.001
all_theta = oneVsAll(X, y, num_labels, lambda_)

<ipython-input-8-d1ad24806257>:17: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
<ipython-input-8-d1ad24806257>:17: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
<ipython-input-8-d1ad24806257>:17: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
<ipython-input-8-d1ad24806257>:17: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))


In [13]:
print(all_theta)

[[ 1.25466936e-05  2.07256407e-04  2.52492689e-02  5.40272669e-05
   5.81769516e-05  3.10494052e-05  2.13993542e-05  1.77541410e-05
   1.28396125e-05  1.00928213e+00  9.01376601e-05  4.20826844e-05
   2.84953638e-02  1.35393633e-05  1.33440840e-05  2.36450659e-05]
 [-1.33600417e-05 -2.11503034e-04 -2.68779605e-02 -5.75267677e-05
  -6.18848562e-05 -3.30643171e-05 -2.23048396e-05 -1.88183689e-05
  -1.37315509e-05 -1.00910857e+00 -9.60637008e-05 -4.47811451e-05
  -3.28239363e-02 -1.49175225e-05 -1.43745475e-05 -2.52483355e-05]]


In [14]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(calcularSigmoide(X.dot(all_theta.T)), axis = 1)
    return p + 1

In [15]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
# XPrueba = X[10:20, :].copy()
# yPrueba = y[10:20].copy()
XPrueba = X[70:80, :].copy()
yPrueba = y[70:80].copy()
print(XPrueba.shape)
print(len(XPrueba))
XPrueba = np.concatenate((np.ones((10, 1)), XPrueba), axis=1)
print(XPrueba)

p = np.argmax(calcularSigmoide(XPrueba.dot(all_theta.T)), axis = 1)
print(p + 1)

(935, 15)
Precision del conjuto de entrenamiento: 91.23%
(10, 15)
10
[[1.0000e+00 3.5000e+01 2.0130e+03 2.0000e+00 2.0000e+00 2.0000e+00
  3.0000e+00 2.0000e+00 1.0000e+00 1.3800e+03 1.0000e+00 2.0000e+00
  1.6000e+03 1.0000e+00 1.0000e+00 2.0000e+00]
 [1.0000e+00 3.5000e+01 2.0120e+03 4.0000e+00 5.0000e+00 2.0000e+00
  1.0000e+00 2.0000e+00 1.0000e+00 4.0000e+03 4.0000e+00 2.0000e+00
  1.6000e+03 1.0000e+00 1.0000e+00 2.0000e+00]
 [1.0000e+00 3.5000e+01 2.0120e+03 5.0000e+00 5.0000e+00 2.0000e+00
  1.0000e+00 2.0000e+00 1.0000e+00 6.0000e+03 2.0000e+00 3.0000e+00
  2.0000e+03 1.0000e+00 1.0000e+00 2.0000e+00]
 [1.0000e+00 9.0000e+00 2.0190e+03 5.0000e+00 5.0000e+00 2.0000e+00
  2.0000e+00 1.0000e+00 1.0000e+00 6.4370e+03 7.0000e+00 3.0000e+00
  2.5000e+03 1.0000e+00 1.0000e+00 2.0000e+00]
 [1.0000e+00 8.0000e+00 2.0140e+03 4.0000e+00 5.0000e+00 2.0000e+00
  1.0000e+00 2.0000e+00 1.0000e+00 1.4000e+04 3.0000e+00 3.0000e+00
  2.0000e+03 1.0000e+00 1.0000e+00 2.0000e+00]
 [1.0000e+00 9.0